In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import warnings
from warnings import filterwarnings
filterwarnings("ignore")

## Reading Data

In [2]:
df = pd.read_csv("data.csv")
df.head(5)

,age,fnlwgt,educational-num,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,...,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife
0,25,226802,7,0,1,0,0,40,0,0,...,False,False,False,False,False,False,False,True,False,False
1,38,89814,9,1,1,0,0,50,0,0,...,False,False,False,False,True,False,False,False,False,False
2,28,336951,12,1,1,0,0,40,0,1,...,True,False,False,False,True,False,False,False,False,False
3,44,160323,10,0,1,7688,0,40,0,1,...,False,False,False,False,True,False,False,False,False,False
4,18,103497,10,1,0,0,0,30,0,0,...,False,False,False,False,False,False,False,True,False,False


## Classification

In [3]:
# Separate features and target
X = df.drop('income', axis=1)
y = df['income']

# Split the data into training (70%) and test (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = RandomForestClassifier(random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)


Accuracy: 0.8627584794922541
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.93      0.91     11233
           1       0.74      0.64      0.68      3420

    accuracy                           0.86     14653
   macro avg       0.82      0.78      0.80     14653
weighted avg       0.86      0.86      0.86     14653



## Values of fairness in the training dataset

In [4]:
# Calculate P(income=1|gender=1) for the Training set
X1 = X_train.copy()
X1['income'] = y_train
gender_1 = X1[X1['gender'] == 1]
P_1 = (gender_1['income'] == 1).mean()

# Calculate P(income=1|gender=0) for the predictions
gender_0 = X1[X1['gender'] == 0]
P_0 = (gender_0['income'] == 1).mean()

print(f"P(income=1|gender=1): {P_1}")
print(f"P(income=1|gender=0): {P_0}")
zemel_fairness = P_1 - P_0
Disparate_impact = P_0/P_1
print(f"Zemel Fairness =  {zemel_fairness}")
print(f"Disparate Impact =  {Disparate_impact}")

P(income=1|gender=1): 0.30650385154061627
P(income=1|gender=0): 0.11145401640067014
Zemel Fairness =  0.19504983513994611
Disparate Impact =  0.3636300680740413


## Values of fairness in for the model on the test dataset

In [21]:
# Convert the test set to a DataFrame to include the gender column
X_2 = X_test.copy()
X_2['income'] = y_test
X_2['predicted_income'] = y_pred

# Calculate P(income=1|gender=1) for the predictions
gender_1 = X_2[X_2['gender'] == 1]
P_1 = (gender_1['predicted_income'] == 1).mean()

# Calculate P(income=1|gender=0) for the predictions
gender_0 = X_2[X_2['gender'] == 0]
P_0 = (gender_0['predicted_income'] == 1).mean()

print(f"P(predicted income=1|gender=1): {P_1}")
print(f"P(predicted income=1|gender=0): {P_0}")
zemel_fairness = P_1 - P_0
Disparate_impact = P_0/P_1
print(f"Zemel Fairness =  {zemel_fairness}")
print(f"Disparate Impact =  {Disparate_impact}")

P(predicted income=1|gender=1): 0.26178330952866763
P(predicted income=1|gender=0): 0.07524221809936095
Zemel Fairness =  0.1865410914293067
Disparate Impact =  0.2874217544076134


## Omit the sensitive feature

In [6]:
# Separate features and target
X_temp = df.drop(['income','gender'], axis=1)
y = df['income']

# Split the data into training (70%) and test (30%) sets
X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(X_temp, y, test_size=0.3, random_state=42)

# Initialize the classifier
clf_o = RandomForestClassifier(random_state=42)

# Train the classifier
clf_o.fit(X_train_temp, y_train_temp)

# Make predictions on the test set
y_pred = clf_o.predict(X_test_temp)

# Evaluate the classifier
accuracy = accuracy_score(y_test_temp, y_pred)
report = classification_report(y_test_temp, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.863645669828704
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.93      0.91     11233
           1       0.74      0.64      0.69      3420

    accuracy                           0.86     14653
   macro avg       0.82      0.79      0.80     14653
weighted avg       0.86      0.86      0.86     14653



In [7]:
# Convert the test set to a DataFrame to include the gender column
X_2 = X_test.copy()
X_2['income'] = y_test
X_2['predicted_income'] = y_pred

# Calculate P(income=1|gender=1) for the predictions
gender_1 = X_2[X_2['gender'] == 1]
P_1 = (gender_1['predicted_income'] == 1).mean()

# Calculate P(income=1|gender=0) for the predictions
gender_0 = X_2[X_2['gender'] == 0]
P_0 = (gender_0['predicted_income'] == 1).mean()

print(f"P(predicted income=1|gender=1): {P_1}")
print(f"P(predicted income=1|gender=0): {P_0}")
zemel_fairness = P_1 - P_0
Disparate_impact = P_0/P_1
print(f"Zemel Fairness =  {zemel_fairness}")
print(f"Disparate Impact =  {Disparate_impact}")

P(predicted income=1|gender=1): 0.262497449500102
P(predicted income=1|gender=0): 0.07854050711193568
Zemel Fairness =  0.18395694238816634
Disparate Impact =  0.2992048389860838


# Making the Model Fair

In [8]:
y_pred = clf.predict(X_train)
y_pred_proba = clf.predict_proba(X_train)
X_train_fair = X_train.copy()
## Adding the predicted class and its probability to the dataset
X_train_fair['income'] = y_train
X_train_fair['predicted_income'] = y_pred
X_train_fair['probability'] = np.max(y_pred_proba, axis=1)

# Calculating n
X_temp = X_train_fair[X_train_fair['gender'] == 0]
Ss = len(X_temp)
Ss_plus = len(X_temp[X_temp['predicted_income'] == 1])
X_temp = X_train_fair[X_train_fair['gender'] == 1]
Ss_bar = len(X_temp)
Ss_bar_plus = len(X_temp[X_temp['predicted_income'] == 1])

n = ((Ss * Ss_bar_plus) - (Ss_bar * Ss_plus)) / (Ss + Ss_bar)
print('n = ', n)

n =  1478.6234753868202


In [9]:
# Create dataset for gender=1 and predicted_income=1 (CP)
CP = X_train_fair[(X_train_fair['gender'] == 1) & (X_train_fair['predicted_income'] == 1)]
CP_sorted = CP.sort_values(by='probability', ascending=False)

# Create dataset for gender=0 and predicted_income=0 (CD)
CD = X_train_fair[(X_train_fair['gender'] == 0) & (X_train_fair['predicted_income'] == 0)]
CD_sorted = CD.sort_values(by='probability', ascending=True)

In [10]:
n = round(n)
CP_indices = CP_sorted.head(n).index
CD_indices = CD_sorted.head(n).index


# Swap the income values in the original dataset
income_CP = X_train_fair.loc[CP_indices, 'income'].values
income_CD = X_train_fair.loc[CD_indices, 'income'].values

X_train_fair.loc[CP_indices, 'income'] = income_CD
X_train_fair.loc[CD_indices, 'income'] = income_CP


y_train_fair = X_train_fair['income']
X_train_fair = X_train_fair.drop(['income', 'probability', 'predicted_income'], axis=1)

In [11]:
clf_fair = RandomForestClassifier(random_state=42)

# Train the classifier
clf_fair.fit(X_train_fair, y_train_fair)

# Make predictions on the test set
y_pred = clf_fair.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.793079915375691
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.89      0.87     11233
           1       0.57      0.47      0.52      3420

    accuracy                           0.79     14653
   macro avg       0.71      0.68      0.69     14653
weighted avg       0.78      0.79      0.79     14653



In [14]:
# Convert the test set to a DataFrame to include the gender column
X_2 = X_test.copy()
X_2['income'] = y_test
X_2['predicted_income'] = y_pred

# Calculate P(income=1|gender=1) for the predictions
gender_1 = X_2[X_2['gender'] == 1]
P_1 = (gender_1['predicted_income'] == 1).mean()

# Calculate P(income=1|gender=0) for the predictions
gender_0 = X_2[X_2['gender'] == 0]
P_0 = (gender_0['predicted_income'] == 1).mean()

print(f"P(predicted income=1|gender=1): {P_1}")
print(f"P(predicted income=1|gender=0): {P_0}")
zemel_fairness = P_1 - P_0
Disparate_impact = P_0/P_1
print(f"Zemel Fairness =  {zemel_fairness}")
print(f"Disparate Impact =  {Disparate_impact}")

P(predicted income=1|gender=1): 0.1870026525198939
P(predicted income=1|gender=0): 0.20964749536178107
Zemel Fairness =  -0.02264484284188717
Disparate Impact =  1.1210936986012974


# Additional fair training

In [22]:
X = df.drop('income', axis=1)
y = df['income']

# Split the data into training (70%) and test (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

df1 = X_train.copy()
df1['income'] = y_train

df_gender_0_income_1 = df1[(df1['gender'] == 0) & (df1['income'] == 1)]
df_gender_1_income_1 = df1[(df1['gender'] == 1) & (df1['income'] == 1)]

num_samples_gender_0_income_1 = df_gender_0_income_1.shape[0]
num_samples_gender_1_income_1 = df_gender_1_income_1.shape[0]

num_to_augment = num_samples_gender_0_income_1 - num_samples_gender_1_income_1

if num_to_augment > 0:
    augmented_data = df_gender_1_income_1.sample(n=num_to_augment, replace=True)
    df_gender_1_income_1_augmented = pd.concat([df_gender_1_income_1, augmented_data], ignore_index=True)
else:
    df_gender_1_income_1_augmented = df_gender_1_income_1

# Combine the balanced subsets with the other data
df_other = df1[~((df1['gender'] == 0) & (df1['income'] == 1)) & ~((df1['gender'] == 1) & (df1['income'] == 1))]
df_balanced = pd.concat([df_gender_0_income_1, df_gender_1_income_1_augmented, df_other], ignore_index=True)

# Split back into x_train and y_train
x_train_balanced = df_balanced.drop(columns=['income'])
y_train_balanced = df_balanced['income']

In [18]:
clf1 = RandomForestClassifier(random_state=42)

# Train the classifier
clf1.fit(x_train_balanced, y_train_balanced)

# Make predictions on the test set
y_pred = clf1.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.8648058418071385
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.93      0.91     11233
           1       0.75      0.64      0.69      3420

    accuracy                           0.86     14653
   macro avg       0.82      0.79      0.80     14653
weighted avg       0.86      0.86      0.86     14653



In [23]:
# Convert the test set to a DataFrame to include the gender column
X_2 = X_test.copy()
X_2['income'] = y_test
X_2['predicted_income'] = y_pred

# Calculate P(income=1|gender=1) for the predictions
gender_1 = X_2[X_2['gender'] == 1]
P_1 = (gender_1['predicted_income'] == 1).mean()

# Calculate P(income=1|gender=0) for the predictions
gender_0 = X_2[X_2['gender'] == 0]
P_0 = (gender_0['predicted_income'] == 1).mean()

print(f"P(predicted income=1|gender=1): {P_1}")
print(f"P(predicted income=1|gender=0): {P_0}")
zemel_fairness = P_1 - P_0
Disparate_impact = P_0/P_1
print(f"Zemel Fairness =  {zemel_fairness}")
print(f"Disparate Impact =  {Disparate_impact}")

P(predicted income=1|gender=1): 0.26178330952866763
P(predicted income=1|gender=0): 0.07524221809936095
Zemel Fairness =  0.1865410914293067
Disparate Impact =  0.2874217544076134
